<a href="https://colab.research.google.com/github/Ganji-Sandeep-10/Youtube-ChatBot/blob/main/Youtube_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1a - **Indexing** (*Document Ingestion*):

In [3]:
pip install youtube-transcript-api --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.0/485.0 kB 4.9 MB/s eta 0:00:00


In [14]:
from youtube_transcript_api import YouTubeTranscriptApi

video_id = "Gfr50f6ZBvo"

try:
    ytt_api = YouTubeTranscriptApi()
    transcript = ytt_api.fetch(video_id, languages=["en"])
    transcript = transcript.to_raw_data()
    result=" ".join(chunk["text"] for chunk in transcript )
    print(result)  # first 3 lines as dicts
except Exception as e:
    print("Error:", e)


the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal question am i an ai program you wrote to interview people until i get good enough 

Step 1b - Indexing (Text Splitting)

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.create_documents([result])
print(len(texts))

168


In [16]:
print(texts)

[Document(metadata={}, page_content="the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal question am i an ai program you wrote to inte

In [7]:
%pip install --upgrade --quiet  langchain-google-genai langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 9.0 MB/s eta 0:0

In [8]:
!pip install -qU sentence-transformers langchain-community langchain-chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [17]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# Initialize HuggingFace embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Initialize Chroma with persistence
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

for d in texts:
    if d.metadata is None:
        d.metadata = {}

vector_store.add_documents(texts)

['bf35cd91-1aaf-4e32-ad4c-aba55bd3f5b6',
 '4c161c3d-78a1-4451-a8a7-1f61dfdd70a4',
 '2683a25d-0376-416f-a57e-dac8641471c9',
 '2e6f4d4b-84be-44ec-a5f2-44ad1048794c',
 'ec58a4b0-93c0-4fe2-a656-47b0d39e35fb',
 'f7548664-588e-478d-83c3-646ea84e5860',
 '0f45b968-db89-47b9-9d24-7272d4086949',
 '371c6d24-ca13-47c5-bd21-e75c754f346b',
 '6389ef81-8abd-4af8-a2f3-7d3ec7c485df',
 '641221c5-58cb-440a-b69a-5d54c1873905',
 'dd3d3c6a-f04e-49f1-be54-3a5c2f27bd04',
 '1cfce45c-6422-45f8-9e7e-b399ee514286',
 'de6a4ccc-071c-4b12-80f6-40832ed18694',
 '4693d2fd-b56c-4632-a130-4658285b51f8',
 '7e8256d4-b5f1-41a9-b855-459ac3cac830',
 '2538ede4-bafa-4ffb-926a-6ced30e0b8cd',
 '7d0c986b-6011-46d4-999f-f9d3c626b346',
 'd608c3ba-1b1a-4b7b-a95b-05fdf5d95172',
 '7389a021-5d9e-4349-b108-5bc59af7cc39',
 'f055844d-849b-4e6c-a2db-c1a84159ff93',
 '75271831-e0d7-4e8a-bb23-2f9314af2d5e',
 '2a1afcae-128a-40a5-8ef9-678c4a90ba9c',
 'e8b8a179-8b6d-49d8-9903-9038609af0fd',
 'b658ca64-bef0-4944-b561-c6160650577a',
 '49657c0f-1571-

In [18]:
# Access underlying Chroma client
collection = vector_store._client.get_collection("example_collection")

# Retrieve by ID (raw, no Pydantic validation)
raw = collection.get(ids=['c4698a3e-b61e-4540-83b0-1c9980e59ad0'])
print(raw)


{'ids': ['c4698a3e-b61e-4540-83b0-1c9980e59ad0'], 'embeddings': None, 'documents': ["deeper maybe simpler explanation yes of things right than the standard model of physics which we know doesn't work but we still keep adding to so um and and that's how i think the beginning of an explanation would look and it would start encompassing many of the mysteries that we have wondered about for thousands of years like you know consciousness uh life and gravity all of these things yeah giving us a glimpses of explanations for those things yeah well um damas dear one of the special human beings in this giant puzzle of ours and it's a huge honor that you would take a pause from the bigger puzzle to solve this small puzzle of a conversation with me today it's truly an honor and a pleasure thank you thank you i really enjoyed it thanks lex thanks for listening to this conversation with demas establish to support this podcast please check out our sponsors in the description and now let me leave you 

Step 2 - Retrieval

In [35]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 4,           # how many to return
        "fetch_k": 20,    # how many candidates to pull before re-ranking
        "lambda_mult": 0.9  # balance relevance vs diversity (0 = max diversity, 1 = max relevance)
    }
)

In [36]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7c76672dda90>, search_type='mmr', search_kwargs={'k': 4, 'fetch_k': 20, 'lambda_mult': 0.9})

In [37]:
retriever.invoke('What is deepmind')

[Document(id='b7c8e6ca-e926-4ec1-9f8b-79646dcfd492', metadata={}, page_content="and how it works this is tough to uh ask you this question because you probably will say it's everything but let's let's try let's try to think to this because you're in a very interesting position where deepmind is the place of some of the most uh brilliant ideas in the history of ai but it's also a place of brilliant engineering so how much of solving intelligence this big goal for deepmind how much of it is science how much is engineering so how much is the algorithms how much is the data how much is the hardware compute infrastructure how much is it the software computer infrastructure yeah um what else is there how much is the human infrastructure and like just the humans interact in certain kinds of ways in all the space of all those ideas how much does maybe like philosophy how much what's the key if um uh if if you were to sort of look back like if we go forward 200 years look back what was the key 

Step 3 - Augmentation

In [53]:
from google.colab import userdata

# Retrieve the API key you saved in Colab
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
import os
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY


In [99]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro")

In [113]:
from langchain_core.prompts import PromptTemplate
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
You are a helpful assistant for answering questions based only on a transcript.

## Instructions:
- Use ONLY the information provided in the context below.
- If the context does not contain the answer, respond with: "I don’t know based on the context provided."
- Do not make up information or rely on outside knowledge.
- Be clear and concise. Use bullet points if multiple facts are relevant.

## Context:
{context}

## Question:
{question}

## Answer (give them a conversational response not a one liners or not in point/subheading style):
""",
    input_variables=["context", "question"]
)


In [101]:
question          = "is the topic of nuclear fusion discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [102]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"and engineering challenges aswell to build these massive fusionreactors and contain the plasma and whatwe try to do whenever we go into a newfieldto apply our systems is we look for umwe talk to domain experts we try andfind the best people in the world tocollaborate with umin this case in fusion we wecollaborated with epfl in switzerlandthe swiss technical institute who areamazing they have a test reactor thatthey were willing to let us use whichyou know i double checked with the teamwe were going to use carefully andsafelyi was impressed they managed to persuadethem to let us use it and um and it's ait's an amazing test reactor they havethere and they try all sorts of prettycrazy experiments on it and um the thethe what we tend to look at is if we gointo a new domain like fusion what areall the bottleneck problems uh likethinking from first principles you knowwhat are all the bottleneck problemsthat are still stopping fusion workingtoday and then we look at we you know weget a\n\nou

In [103]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [104]:
final_prompt

StringPromptValue(text='\nYou are a helpful assistant for answering questions based only on a transcript.\n\n## Instructions:\n- Use ONLY the information provided in the context below.\n- If the context does not contain the answer, respond with: "I don’t know based on the context provided."\n- Do not make up information or rely on outside knowledge.\n- Be clear and concise. Use bullet points if multiple facts are relevant.\n\n## Context:\nand engineering challenges aswell to build these massive fusionreactors and contain the plasma and whatwe try to do whenever we go into a newfieldto apply our systems is we look for umwe talk to domain experts we try andfind the best people in the world tocollaborate with umin this case in fusion we wecollaborated with epfl in switzerlandthe swiss technical institute who areamazing they have a test reactor thatthey were willing to let us use whichyou know i double checked with the teamwe were going to use carefully andsafelyi was impressed they manage

In [105]:
answer = llm.invoke(final_prompt)
print(answer.content)

Yes, the topic of nuclear fusion is discussed. The speaker explains how their team is applying AI to help solve challenges in the field.

Based on the context, here is what was discussed:

*   **Collaboration:** The team collaborated with EPFL in Switzerland (the Swiss technical institute), who they consider to be among the best experts in the world.
*   **Methodology:** They used EPFL's test reactor and a plasma simulator to apply their AI systems to bottleneck problems that are currently stopping fusion from working.
*   **AI Achievement:** They developed an AI controller that successfully held plasma in specific shapes, which is described as being like "carving the plasma." This was held for a record amount of time.
*   **Publication:** This achievement was significant enough to be published in a Nature paper.
*   **Future Plans:** The team is now talking with many fusion startups to identify the next problem they can tackle in the field.
*   **Overall Goal:** The speaker identifies

Building a Chain

In [106]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [107]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [108]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [109]:
parallel_chain.invoke('who is Demis')

{'context': "giving youglimpses ofwhat things you totally missed in thephysics of todayexactly just here here's glimpses of nolike there's a muchuha much more elaborate world or a muchsimpler world or somethinga much deeper maybe simpler explanationyes of things right than the standardmodel of physics which we know doesn'twork but we still keep adding to so umand and that's how i think the beginningof an explanation would look and itwould start encompassing many of themysteries that we have wondered aboutfor thousands of years like you knowconsciousnessuh life and gravity all of these thingsyeah giving us a glimpses ofexplanations for those things yeahwell umdamas dear one of the specialhuman beings in this giant puzzle ofours and it's a huge honor that youwould take a pause from the biggerpuzzle to solve this small puzzle of aconversation with me today it's truly anhonor and a pleasure thank you thank youi really enjoyed it thanks lexthanks for listening to thisconversation with demas

In [110]:
parser = StrOutputParser()

In [114]:
main_chain = parallel_chain | prompt | llm | parser

In [115]:
main_chain.invoke('Can you summarize the video')

"Based on the text provided, the speakers discuss what a more fundamental explanation of the universe might look like, suggesting it would be a deeper and simpler explanation of physics than the standard model, potentially encompassing mysteries like consciousness, life, and gravity. They touch upon the idea that a sign of true intelligence is the ability to explain complex topics simply, referencing Richard Feynman. The conversation also delves into the history of AI and chess, mentioning pioneers like Claude Shannon and Alan Turing, and reflects on the Deep Blue vs. Kasparov match, with one speaker being more impressed by Kasparov's human mind than the computer."